In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5733
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4586:], batch_size=32)
    np.save('5733npy-glove/text-glove_5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-glove/text-glove_5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 85s 18ms/step - loss: 2.5235 - v_loss: 0.7256 - a_loss: 0.8715 - d_loss: 0.9194 - v_ccc: 0.2723 - a_ccc: 0.1256 - d_ccc: 0.0786 - val_loss: 2.6730 - val_v_loss: 0.8427 - val_a_loss: 0.8962 - val_d_loss: 0.9323 - val_v_ccc: 0.1567 - val_a_ccc: 0.1031 - val_d_ccc: 0.0673
Epoch 2/50
4586/4586 [==============================] - 81s 18ms/step - loss: 2.0310 - v_loss: 0.5690 - a_loss: 0.7051 - d_loss: 0.7609 - v_ccc: 0.4301 - a_ccc: 0.2967 - d_ccc: 0.2422 - val_loss: 2.6611 - val_v_loss: 0.8726 - val_a_loss: 0.8906 - val_d_loss: 0.8968 - val_v_ccc: 0.1265 - val_a_ccc: 0.1093 - val_d_ccc: 0.1030
Epoch 3/50
4586/4586 [==============================] - 81s 18ms/step - loss: 1.8181 - v_loss: 0.4997 - a_loss: 0.6306 - d_loss: 0.6860 - v_ccc: 0.4989 - a_ccc: 0.3688 - d_ccc: 0.3142 - val_loss: 2.6144 - val_v_loss: 0

Epoch 11/50
1433/1433 [==============================] - 8s 5ms/step
[0.18888643383979797, 0.17732036113739014, 0.11956151574850082]
1
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 84s 18ms/step - loss: 2.5528 - v_loss: 0.7353 - a_loss: 0.8908 - d_loss: 0.9240 - v_ccc: 0.2627 - a_ccc: 0.1088 - d_ccc: 0.0757 - val_loss: 2.6622 - val_v_loss: 0.8887 - val_a_loss: 0.8841 - val_d_loss: 0.8866 - val_v_ccc: 0.1102 - val_a_ccc: 0.1152 - val_d_ccc: 0.1125
Epoch 2/50
4586/4586 [==============================] - 82s 18ms/step - loss: 2.0527 - v_loss: 0.5799 - a_loss: 0.7047 - d_loss: 0.7645 - v_ccc: 0.4200 - a_ccc: 0.2928 - d_ccc: 0.2344 - val_loss: 2.5511 - val_v_loss: 0.7898 - val_a_loss: 0.8627 - val_d_loss: 0.8975 - val_v_ccc: 0.2097 - val_a_ccc: 0.1368 - val_d_ccc: 0.1025
Epoch 3/50
4586/4586 [==============================] - 82s 18ms/step - loss: 1.8165 - v_loss: 0.5024 - a_loss: 0.6274 - d_loss: 0.6852 - v_ccc: 0.4972 - a_ccc: 0.37

1433/1433 [==============================] - 8s 5ms/step
[0.2507646381855011, 0.22921296954154968, 0.15841582417488098]
3
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 85s 19ms/step - loss: 2.5258 - v_loss: 0.7298 - a_loss: 0.8781 - d_loss: 0.9174 - v_ccc: 0.2707 - a_ccc: 0.1217 - d_ccc: 0.0818 - val_loss: 2.6933 - val_v_loss: 0.8046 - val_a_loss: 0.9432 - val_d_loss: 0.9448 - val_v_ccc: 0.1954 - val_a_ccc: 0.0559 - val_d_ccc: 0.0554
Epoch 2/50
4586/4586 [==============================] - 81s 18ms/step - loss: 2.0441 - v_loss: 0.5751 - a_loss: 0.7067 - d_loss: 0.7622 - v_ccc: 0.4244 - a_ccc: 0.2929 - d_ccc: 0.2387 - val_loss: 2.6336 - val_v_loss: 0.7981 - val_a_loss: 0.9045 - val_d_loss: 0.9312 - val_v_ccc: 0.2016 - val_a_ccc: 0.0954 - val_d_ccc: 0.0695
Epoch 3/50
4586/4586 [==============================] - 81s 18ms/step - loss: 1.8182 - v_loss: 0.5015 - a_loss: 0.6333 - d_loss: 0.6829 - v_ccc: 0.4979 - a_ccc: 0.3671 - d_ccc: 0

4586/4586 [==============================] - 82s 18ms/step - loss: 0.7678 - v_loss: 0.2204 - a_loss: 0.2714 - d_loss: 0.2784 - v_ccc: 0.7796 - a_ccc: 0.7297 - d_ccc: 0.7229 - val_loss: 2.6934 - val_v_loss: 0.8246 - val_a_loss: 0.9178 - val_d_loss: 0.9487 - val_v_ccc: 0.1737 - val_a_ccc: 0.0821 - val_d_ccc: 0.0508
Epoch 13/50
1433/1433 [==============================] - 8s 5ms/step
[0.19742244482040405, 0.22915072739124298, 0.11872478574514389]
5
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 85s 19ms/step - loss: 2.5681 - v_loss: 0.7399 - a_loss: 0.8910 - d_loss: 0.9353 - v_ccc: 0.2575 - a_ccc: 0.1088 - d_ccc: 0.0656 - val_loss: 2.5222 - val_v_loss: 0.8082 - val_a_loss: 0.8547 - val_d_loss: 0.8581 - val_v_ccc: 0.1914 - val_a_ccc: 0.1450 - val_d_ccc: 0.1413
Epoch 2/50
4586/4586 [==============================] - 80s 18ms/step - loss: 2.0711 - v_loss: 0.5784 - a_loss: 0.7206 - d_loss: 0.7721 - v_ccc: 0.4217 - a_ccc: 0.2803 - d_ccc:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 80s 17ms/step - loss: 2.0794 - v_loss: 0.5792 - a_loss: 0.7274 - d_loss: 0.7779 - v_ccc: 0.4210 - a_ccc: 0.2752 - d_ccc: 0.2245 - val_loss: 2.6596 - val_v_loss: 0.7978 - val_a_loss: 0.9196 - val_d_loss: 0.9424 - val_v_ccc: 0.2020 - val_a_ccc: 0.0804 - val_d_ccc: 0.0579
Epoch 3/50
4586/4586 [==============================] - 79s 17ms/step - loss: 1.8387 - v_loss: 0.5097 - a_loss: 0.6327 - d_loss: 0.6922 - v_ccc: 0.4896 - a_ccc: 0.3660 - d_ccc: 0.3056 - val_loss: 2.8088 - val_v_loss: 0.8862 - val_a_loss: 0.9333 - val_d_loss: 0.9885 - val_v_ccc: 0.1127 - val_a_ccc: 0.0665 - val_d_ccc: 0.0120
Epoch 4/50
4586/4586 [==============================] - 80s 17ms/step - loss: 1.6253 - v_loss: 0.4533 - a_loss: 0.5699 - d_loss: 0.6072 - v_ccc: 0.5488 - a_ccc: 0.4316 - d_ccc: 0.3943 - val_loss: 2.6876 - val_v_loss: 0.8490 - val_a_loss: 0.8948 - val_d_loss: 0.9440 - val_v_ccc: 0.1510 - val_a_ccc: 0.1052 - val_d_ccc: 0.0562
Epoch 5/50
4586/4586 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 81s 18ms/step - loss: 0.9878 - v_loss: 0.2784 - a_loss: 0.3491 - d_loss: 0.3606 - v_ccc: 0.7215 - a_ccc: 0.6509 - d_ccc: 0.6398 - val_loss: 2.6462 - val_v_loss: 0.8524 - val_a_loss: 0.8749 - val_d_loss: 0.9170 - val_v_ccc: 0.1463 - val_a_ccc: 0.1249 - val_d_ccc: 0.0826
Epoch 10/50
4586/4586 [==============================] - 81s 18ms/step - loss: 0.8916 - v_loss: 0.2554 - a_loss: 0.3103 - d_loss: 0.3272 - v_ccc: 0.7451 - a_ccc: 0.6901 - d_ccc: 0.6733 - val_loss: 2.6132 - val_v_loss: 0.8361 - val_a_loss: 0.8736 - val_d_loss: 0.9022 - val_v_ccc: 0.1630 - val_a_ccc: 0.1264 - val_d_ccc: 0.0974
Epoch 11/50
4586/4586 [==============================] - 81s 18ms/step - loss: 0.8088 - v_loss: 0.2375 - a_loss: 0.2765 - d_loss: 0.2946 - v_ccc: 0.7631 - a_ccc: 0.7237 - d_ccc: 0.7044 - val_loss: 2.7114 - val_v_loss: 0.8529 - val_a_loss: 0.9058 - val_d_loss: 0.9509 - val_v_ccc: 0.1459 - val_a_ccc: 0.0939 - val_d_ccc: 0.0488
Epoch 12/50
1433/1433 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 80s 17ms/step - loss: 1.8186 - v_loss: 0.4976 - a_loss: 0.6370 - d_loss: 0.6830 - v_ccc: 0.5034 - a_ccc: 0.3627 - d_ccc: 0.3154 - val_loss: 2.6510 - val_v_loss: 0.8271 - val_a_loss: 0.8973 - val_d_loss: 0.9243 - val_v_ccc: 0.1714 - val_a_ccc: 0.1020 - val_d_ccc: 0.0755
Epoch 4/50
4586/4586 [==============================] - 80s 18ms/step - loss: 1.6314 - v_loss: 0.4489 - a_loss: 0.5720 - d_loss: 0.6108 - v_ccc: 0.5510 - a_ccc: 0.4286 - d_ccc: 0.3891 - val_loss: 2.6886 - val_v_loss: 0.8191 - val_a_loss: 0.9060 - val_d_loss: 0.9627 - val_v_ccc: 0.1804 - val_a_ccc: 0.0934 - val_d_ccc: 0.0375
Epoch 5/50
4586/4586 [==============================] - 80s 18ms/step - loss: 1.4625 - v_loss: 0.4068 - a_loss: 0.5147 - d_loss: 0.5385 - v_ccc: 0.5927 - a_ccc: 0.4842 - d_ccc: 0.4607 - val_loss: 2.6697 - val_v_loss: 0.8318 - val_a_loss: 0.8988 - val_d_loss: 0.9363 - val_v_ccc: 0.1666 - val_a_ccc: 0.1003 - val_d_ccc: 0.0634
Epoch 6/50
4586/4586 [===========

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)